### Simple Function for Taking a Dictionary of Pandas DataFrames and Outputing as Excel File

So the following script is pretty straightforward. We have an end-user, and they like their data in an excel notebook where the data is separated into multiple sheets.

Lets start by grabbing some data.

In [1]:
# ---------------------------------------- [ LIBRARY IMPORT ] -----
import pandas as pd
import os
import re

# ---------------------------------------- [ DATA IMPORT ] -----
# --- lets grab some sample data this is electric vehicle data from washington state
elec_vehicle_url = "https://data.wa.gov/api/views/f6w7-q2d2/rows.csv?accessType=DOWNLOAD"

# --- this is pretty big so its gonna take a bit of time
elec_vehicle_df = pd.read_csv(elec_vehicle_url)

# --- im picky about column names
def name_repair(StrInput):
    '''
    Parameters
    ----------
    StrInput : TYPE string
        DESCRIPTION. takes a string with capital letters and imposes underscores and then lowers it

    Returns
    -------
    str_out : TYPE string
        DESCRIPTION. the new string we want
    '''
    str_out = re.sub('[^a-zA-Z]', ' ', StrInput).lower()
    str_out = str_out.strip()
    str_out = re.sub(' {1,}', ' ', str_out)
    str_out = re.sub(' ', '_', str_out)
    return str_out

# --- fix those columns
elec_vehicle_df.columns = [name_repair(i) for i in elec_vehicle_df.columns]
print(elec_vehicle_df)

               vin     county      city state  postal_code  model_year  \
0       5YJ3E1EA5L       King   Seattle    WA      98133.0        2020   
1       5UX43EU08R       King   Seattle    WA      98125.0        2024   
2       5UX43EU06R       King   Seattle    WA      98102.0        2024   
3       5YJ3E1EA1J       King  Kirkland    WA      98034.0        2018   
4       1G1RA6E43C   Thurston   Olympia    WA      98501.0        2012   
...            ...        ...       ...   ...          ...         ...   
220220  KNDC4DLC5N   Thurston   Olympia    WA      98501.0        2022   
220221  5YJ3E1EA5J  Snohomish   Bothell    WA      98012.0        2018   
220222  5YJ3E1EB0K  Snohomish   Bothell    WA      98012.0        2019   
220223  2C4RC1L72N       King   Redmond    WA      98053.0        2022   
220224  5YJ3E1EA6J       King  Bellevue    WA      98007.0        2018   

             make     model                   electric_vehicle_type  \
0           TESLA   MODEL 3          Bat

Now lets aggregate it and get the count of registered vehicles by make, city and county

We are then going to split it by county into a dictionary so each key of the dictionary is a county and its corresponding value is the dataframe

In [2]:
# ---------------------------------------- [ DATA AGGREGATE ] -----
# --- get vehicle counts by make city and county
elec_vehicle_agg = elec_vehicle_df[['make', 'city', 'county', 'vin']].groupby(['make', 'city', 'county']).count().reset_index()
elec_vehicle_agg = elec_vehicle_agg.sort_values(by = ['county', 'city', 'make']).reset_index(drop = True)

# --- dictionary to append to
df_dct = {}

# --- split by county
for county in elec_vehicle_agg['county'].unique():
    tmp_df = elec_vehicle_agg[elec_vehicle_agg['county'] == county]
    tmp_df = tmp_df[['city', 'make', 'vin']]
    tmp_df.columns = ['city', 'make', 'vehicle_count']
    df_dct[county] = tmp_df

Heres what you came for - the function for taking a dictionary of data frames and outputing as an excel file

In [3]:
# ---------------------------------------- [ EXPORT IN EXCEL ] -----
# --- export function
def df_dict_xl(df_dict, xl_file_name, col_buff = 2):
    
    # --- function returns length of columns to set length to
    def get_col_len(dfx):
        # define list to append to
        col_len_lst = []
        # declare list of column names
        col_lst = [col for col in dfx.columns]
        # obtain max char count for each column
        for i in range(len(col_lst)):
            col_len_lst.append(max(list(dfx[col_lst[i]].astype(str).str.len()) + [len(col_lst[i]) * 1.25]))
        # return col length list
        return(col_len_lst)
    
    # --- define excel writer
    writer = pd.ExcelWriter(xl_file_name, engine='xlsxwriter')
    
    # --- itterate to find list of sheet names and data found in dictionary
    sheet_name_lst = [key for key in df_dict]
    df_lst = [df_dict.get(i) for i in sheet_name_lst]
    
    # --- loop through list of sheet names to get custom col width
    for i in range(len(sheet_name_lst)):
        # save data to work sheet
        df_lst[i].to_excel(writer, sheet_name = sheet_name_lst[i], index = False)
        # define sheet to modify column lengths
        worksheet = writer.sheets[sheet_name_lst[i]]
        # define col lengths for each col in data frame
        df_col_len = get_col_len(df_lst[i])
        # loop through columns and modify column length
        for i in range(len(df_col_len)):
            worksheet.set_column(i, i, (df_col_len[i] + col_buff))

    # --- save workbook
    writer.save()

Its a pretty straightforward function to call. Lets output the data and check out the results.

In [4]:
# --- final step
df_dict_xl(df_dct, 'D:\elec_vehicles.xlsx', 3)